###**`SCRAPE REVIEWS FROM WEB`**

`https://www.consumeraffairs.com/automotive/aaa.html`

In [1]:
#@title 

# !pip install pandas requests BeautifulSoup4 
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

# pd.set_option(
#     'max_colwidth', None,
#     'max_rows', None,
# )


def flatList(nestedList):
  for idx,i in enumerate(nestedList):
    if isinstance(i, list):
      nestedList = sum( [nestedList.pop(idx)], nestedList )
      return flatList(nestedList)
  return nestedList




def scraper(base_url):
  # base_url = 'https://www.consumeraffairs.com/automotive/aaa.html'

  allRevs = []
  allRatings = []
  pagewise_reviews = []
  pagewise_ratings = []
  all_pages_reviews =[]

  for i in range(2,69):
    # print(f'PROCESSING PAGE: {i}')
    query_parameter = '?page='+str(i)+'#scroll_to_reviews=true'

    url = base_url + query_parameter
    response = requests.get(url)
    soup = bs(response.content, 'html.parser') 
    rev_div = soup.findAll("div", attrs={"class","rvw-bd"})
    rating_div = soup.findAll("div",attrs={"class","rvw__hdr"})
    allRevs.append( rev_div )
    allRatings.append( rating_div )

  allRevs = flatList( allRevs )
  allRatings = flatList( allRatings)
  
#     setRevs = set( allRevs )
#     setRatings = set( allRatings)

  for z in range(len(allRevs)-0 ):
    myReview = allRevs[z].find("p").text
    pagewise_reviews.append( myReview )

  for z in range(len(allRatings)-0):
    myRating = allRatings[z].find("meta", itemprop = "ratingValue")
    #print(f'Rating: {myRating["content"] }')

    try:
      pagewise_ratings.append( myRating['content'] )
    except:
      pagewise_ratings.append( '•' )

    
    # if myRating['content'] is None:
    #   pagewise_ratings.append('•')
    # else:
    #   pagewise_ratings.append( myRating['content'] )



  fullList = flatList(pagewise_reviews)

  fullRatingList = flatList(pagewise_ratings)

  # print(len(fullRatingList))
  # print(len(fullList))

  return [fullList,fullRatingList]


base_url = 'https://www.consumeraffairs.com/automotive/aaa.html'

scrapedData = scraper(base_url)


reviews = scrapedData[0]
ratings = scrapedData[1]


df = pd.DataFrame(
    data= list(zip(reviews,ratings)),
    columns=['reviews','ratings']
)


dfMISC = df[ df['ratings'] == '•' ]

df = df[ df['ratings'] != '•' ]

df['ratings'] = df['ratings'].astype(int)

posDF = df.query('ratings >= 3', engine='python')
negDF = df.query('ratings <= 2', engine='python')

print('TOTAL REVIEWS:', len(df) )
print('POSITIVE REVIEWS (star >= 3):', str(len(posDF)).ljust(4), ' • ',  str(100*round(len(posDF)/len(df), 2)) + '%'        )
print('NEGATIVE REVIEWS (star <= 2):', str(len(negDF)).ljust(4), ' • ',  str(100*round(len(negDF)/len(df), 2)) + '%'        )

TOTAL REVIEWS: 1867
POSITIVE REVIEWS (star >= 3): 204   •  11.0%
NEGATIVE REVIEWS (star <= 2): 1663  •  89.0%



___



###**`COMPUTE LDA FOR REVIEWS`**

In [3]:
#@title
import warnings
warnings.filterwarnings("ignore")

# python3 -m spacy download en #Language model
!pip3 install -q spacy

# For topic modeling
!pip3 install -q gensim 

# For visualizing topic models
!pip3 install -q pyLDAvis 

import sys
!{sys.executable} -m pip install -q --user PyPDF2==1.26.0  gensim==3.8.3 nltk==3.5 
!{sys.executable} -m pip install -q --user pyLDAvis==2.1.2 wordcloud pyspark PyArrow pandas delta-spark mlflow PyPDF2

#stopwords from NLTK.
import nltk
nltk.download('stopwords', quiet=True)


import re
import numpy as np
import pandas as  pd
from pprint import pprint# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel# spaCy for preprocessing
import spacy# Plotting tools
import pyLDAvis

fork = None
try:
  import pyLDAvis.gensim
  fork=1
except:
  import pyLDAvis.gensim_models
  fork=2

import matplotlib.pyplot as plt

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")


### ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- 

# NLTK Stop words
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

# Convert to list 
data = negDF.reviews.values.tolist()  

# Remove distracting single quotes 
data = [re.sub("\'", "", sent) for sent in data]  


# pprint(data[:1])


def sent_to_words(sentences):
  for sentence in sentences:
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))            #deacc=True removes punctuations

data_words = list(sent_to_words(data))
# print(' • ')
# print(data_words[:1])

# Build the bigram and trigram models
bigram  = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod  = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
# print('TRIGRAM', trigram_mod[bigram_mod[data_words[0]]]  )




# Define function for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out


# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# print('lemmatized'.upper(), data_lemmatized[:1])


# Create Dictionary 
id2word = corpora.Dictionary(data_lemmatized)  

# Create Corpus 
texts = data_lemmatized  

# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts]  

# View 
# print('CORPUS',corpus[:1])
# print(f'READABLE CORPUS: {[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]}')

print()

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

# Print the keyword of topics
# print('lda_model.print_topics()')
# pprint( lda_model.print_topics() )
doc_lda = lda_model[corpus]

# Compute Perplexity
#### a measure of how good the model is. lower the better.
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

print('\n'*2)

# Visualize the topics
pyLDAvis.enable_notebook()

if fork==1:
  vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
else:
  vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)

vis

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable




Perplexity:  -6.828961596613735

Coherence Score:  0.48625749268759205





PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.287954 -0.043796       1        1  55.749483
1     -0.323909  0.180536       2        1  18.769957
7     -0.189462  0.040455       3        1  12.467635
9     -0.067162 -0.372867       4        1   8.281113
3      0.144805  0.057043       5        1   1.623917
4      0.148813  0.044069       6        1   1.263090
5      0.149115  0.031341       7        1   1.026829
2      0.142345  0.021414       8        1   0.327450
6      0.145376  0.024240       9        1   0.301322
0      0.138033  0.017566      10        1   0.189204, topic_info=            Term         Freq        Total Category  logprob  loglift
4           call  2588.000000  2588.000000  Default  30.0000  30.0000
13          hour   940.000000   940.000000  Default  29.0000  29.0000
19    membership   600.000000   600.000000  Default  28.0000  28.0000
0            aaa  2723.000000  2723.000000  Default  27.0000  27.0000
74          tell  1336.000000  1336.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
2533    horribly     0.031201     0.712153  Topic10  -8.4383   3.1422
2532   dehydrate     0.031201     0.712153  Topic10  -8.4383   3.1422
2538       vomit     0.031201     0.712153  Topic10  -8.4383   3.1422
2534        sand     0.031201     0.712153  Topic10  -8.4383   3.1422
2536    sunburnt     0.031201     0.712153  Topic10  -8.4383   3.1422

[423 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
0         1  0.993096         aaa
0         2  0.002937         aaa
0         3  0.002937         aaa
0         4  0.000734         aaa
488       8  0.902895  absolutely
...     ...       ...         ...
319       1  0.997792        work
343       1  0.027976       wrong
343       3  0.960494       wrong
190       1  0.898816        year
190       4  0.101223        year

[435 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 2, 8, 10, 4, 5, 6, 3, 7, 1])